Can Do Basic Segmentation on Sticked Dentin

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
# 計算基於 ['enamel_x'] 和 ['enamel_y'] 的距離函數
def calculate_distance(row_true, row_cleaned):
    true_values = np.array([row_true['enamel_x'], row_true['enamel_y']])
    cleaned_values = np.array([row_cleaned['enamel_x'], row_cleaned['enamel_y']])
    return np.linalg.norm(true_values - cleaned_values)

# 此函數使用影像的輪廓來計算物體的旋轉角度，確保物體的長邊垂直
def get_rotation_angle(mask):
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return 0
    cnt = contours[0]

    # Get rotated rectangle from the largest contour
    rect = cv2.minAreaRect(cnt)
    angle = rect[2]

    #print(angle)
    # Ensure the longest side is vertical
    if rect[1][0] > rect[1][1]:
        angle += 90
    if angle > 90:
        angle += 180
    return angle

# 此函數根據給定的角度旋轉圖像，保持中心點不變。
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))

    return rotated

# 此函數將旋轉後的座標轉換回旋轉前的原始座標。
def convert_coord_back(coord, angle, image):
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    
    # Create the inverse rotation matrix
    M_inv = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    
    # Add a one to the coordinate for matrix multiplication
    coord_ones = np.array([coord[0], coord[1], 1.0])
    
    # Perform the inverse rotation
    original_coord_back = M_inv.dot(coord_ones)
    
    original_coord_back = original_coord_back[:2].astype(int)
    
    return original_coord_back

# 此函數判斷某個值是否位於目標範圍內
def is_within_range(value, target, range_size=50):
    return target - range_size <= value <= target + range_size

# 負責當其中一側的 dentin 值為 None 時，自動分配另一側的值，以避免缺失資料
def assign_non_none_values(dentin_left_x, dentin_left_y, dentin_right_x, dentin_right_y):
    # 检查 dentin_left_x 和 dentin_right_x
    if dentin_left_x is None and dentin_right_x is not None:
        dentin_left_x = dentin_right_x
    elif dentin_right_x is None and dentin_left_x is not None:
        dentin_right_x = dentin_left_x

    # 检查 dentin_left_y 和 dentin_right_y
    if dentin_left_y is None and dentin_right_y is not None:
        dentin_left_y = dentin_right_y
    elif dentin_right_y is None and dentin_left_y is not None:
        dentin_right_y = dentin_left_y

    return dentin_left_x, dentin_left_y, dentin_right_x, dentin_right_y

#　此函數從輪廓中選取根據 y 坐標排序的前 100 個點，用來定位牙齒或組織的特定區域
def get_top_100_points(contours, reverse=True):
    all_points = []
    for contour in contours:
        sorted_points = sorted(contour, key=lambda x: x[0][1], reverse=reverse)
        top_points = sorted_points
        all_points.extend(top_points)
    # Sort all points and get the top 10
    all_points = sorted(all_points, key=lambda x: x[0][1], reverse=reverse)
    return all_points


# 计算 true_stage
def calculate_true_stage(row):
    enamel_x, enamel_y = row['enamel_x'], row['enamel_y']
    gum_x, gum_y = row['gum_x'], row['gum_y']
    dentin_x, dentin_y = row['dentin_x'], row['dentin_y']
    
    # 计算 A, B, C 點之間的距離
    AB = np.sqrt((enamel_x - gum_x) ** 2 + (enamel_y - gum_y) ** 2)
    AC = np.sqrt((enamel_x - dentin_x) ** 2 + (enamel_y - dentin_y) ** 2)
    
    # 计算 percentage
    percentage = (AB / AC) * 100
    
    # 判定 true_stage
    if percentage < 15:
        stage = "I"
    elif 15 <= percentage <= 33:
        stage = "II"
    else:
        stage = "III"
    
    return stage
               
# 計算 percentage 和期數
def calculate_predicted_stage(row):
    enamel_x, enamel_y = row['enamel_x'], row['enamel_y']
    gum_x, gum_y = row['gum_x'], row['gum_y']
    dentin_x, dentin_y = row['dentin_x'], row['dentin_y']
    
    # 計算 A, B, C 點之間的距離
    AB = np.sqrt((enamel_x - gum_x) ** 2 + (enamel_y - gum_y) ** 2)
    AC = np.sqrt((enamel_x - dentin_x) ** 2 + (enamel_y - dentin_y) ** 2)
    
    # 計算 percentage
    percentage = (AB / AC) * 100
    
    # 判斷期數
    if percentage < 15:
        stage = "I"
    elif 15 <= percentage <= 33:
        stage = "II"
    else:
        stage = "III"
    
    return percentage, stage   

In [3]:
# def split_erase():
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

Side_Shift = 3
Side_Contou_THD = 100
Match_Shift = 10
Match_Dist = 50

Window_Group = 100
Slide_Group = 4

def get_end_point(mask_sobj_in, flag_dir = 0):
    
    y_arr, x_arr = np.nonzero(mask_sobj_in)
    
    sorted_indx = np.argsort(y_arr)
    
    if(flag_dir == 0):#Top
        return x_arr[sorted_indx[-1]], y_arr[sorted_indx[-1]]
        
    else:#Bottom
        return x_arr[sorted_indx[0]], y_arr[sorted_indx[0]]

def mask_divide(pair_rect_x, mask_inter_in, rdentin_bin):
    
    mask_divide = np.copy(rdentin_bin)
    
    i = 0
    while(i < len(pair_rect_x)):
        st = pair_rect_x[i]
        
        # Copy and clear outoff rect.
        mask_tmp = np.copy(mask_inter_in)
        mask_tmp[:, 0:st] = 0
        mask_tmp[:, st + Window_Group:] = 0
        
        coordinates = np.argwhere(mask_tmp == 255)
#         coord_list = [tuple(coord) for coord in coordinates]
        
#         kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(coord_list)
        
#         print(kmeans.cluster_centers_)
#         print(type(kmeans.cluster_centers_))

        # start_point = (int(kmeans.cluster_centers_[0][1]), int(kmeans.cluster_centers_[0][0]))
        # end_point = (int(kmeans.cluster_centers_[1][1]), int(kmeans.cluster_centers_[1][0]))
        
        
        
        # 使用 DBSCAN 進行聚類
        dbscan = DBSCAN(eps=15, min_samples=10)  # 調整 eps 和 min_samples 根據數據密度
        dbscan_labels = dbscan.fit_predict(coordinates)

        # 確保有至少兩個有效簇
        unique_labels = set(dbscan_labels) - {-1}  # 排除噪聲 (-1 為噪聲標籤)
        if len(unique_labels) < 2:
            print("DBSCAN 未能找到兩個有效的簇，請調整 eps 或 min_samples")
            return mask_divide

        # 計算每個簇的中心點
        cluster_centers = []
        cluster_centers_x = []
        for label in unique_labels:
            cluster_points = coordinates[dbscan_labels == label]
            center = np.mean(cluster_points, axis=0)  # 計算簇中心
            cluster_centers.append(center)
            cluster_centers_x.append(center[0])

        # 確保有兩個簇中心點
        if len(cluster_centers) > 2:
             print("聚類結果 > 2")

        # sort_indx = np.argsort(cluster_centers_x)  
            
        # 將中心點轉換為整數
        start_point = (int(cluster_centers[0][1]), int(cluster_centers[0][0]))
        end_point = (int(cluster_centers[1][1]), int(cluster_centers[1][0]))

        print(f"Start Point: {start_point}, End Point: {end_point}")
        
        cv2.line(mask_divide, start_point, end_point, 0, 4)
        cv2.line(rdentin_bin, start_point, end_point, 0, 4)
    
        i += 1
    
    return mask_divide
    
def get_pair_rect(mask_inter_in, mask_org):
    
    mask_inter_cp = np.copy(mask_inter_in)
    group_inter = np.zeros_like(mask_inter_cp)
    width = mask_inter_cp.shape[1]
    
    st_x = 0
    ed_x = st_x + Window_Group
    list_can_x = [] # candidator
    list_can_area = [] #candidator
    find_flag = 0 # If find one pair
    pair_rect_st_x = []
    while (ed_x < width):
        mk_win = mask_inter_cp[:, st_x:ed_x]
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mk_win, connectivity=8)
        
        print(num_labels)
        
        if num_labels <= 2:
            if find_flag == 1:
                find_flag = 0
               
                sort_indx = np.argsort(list_can_area)
                can_st_x = list_can_x[sort_indx[-1]]
                pair_rect_st_x.append(can_st_x)
                
                #Clear intersections for avoiding repeat rect.
                mask_inter_cp[:, can_st_x:can_st_x+Window_Group] = 0
                
                list_can_x.clear()
                list_can_area.clear()
            
            
            st_x += Slide_Group
            ed_x = st_x + Window_Group    
                    
            continue
        
        else:
            find_flag = 1

            tot_area = 0
            for i in range(1, num_labels):
                tot_area += stats[i, cv2.CC_STAT_AREA]        
            list_can_area.append(tot_area)
            list_can_x.append(st_x)

            
            st_x += Slide_Group
            ed_x = st_x + Window_Group

            # print('find ' + str(st_x))
        
        
    print('find a pair rect. st. x = {}'.format(pair_rect_st_x))
    
    
    return pair_rect_st_x
    
    
    
def mask_intersec(mask_lt_in, mask_rt_in, rec_y, rec_h, comp_mask_in):
    mk_lt = np.copy(mask_lt_in)
    mk_rt = np.copy(mask_rt_in)
    comp_mask = np.copy(comp_mask_in)
    
    # Use WarpAffine for x axis shift
    height, width = mk_lt.shape[:2]
    M = np.float32([[1, 0, Match_Shift], [0, 1, 0]])
    mk_lt_sf = cv2.warpAffine(mk_lt, M, (width, height))
    M = np.float32([[1, 0, -Match_Shift], [0, 1, 0]])
    mk_rt_sf = cv2.warpAffine(mk_rt, M, (width, height))
    
    mask_inter = cv2.bitwise_and(mk_lt_sf, mk_rt_sf)
    
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_inter, connectivity=8)
    intersec_id = 1
    for i in range(1, num_labels):
        if centroids[i][1] < rec_y + 0.5*rec_h:
                continue
        
    return mask_inter
    
    
#Filter Noise Components
def process_mask(mask_in):
    mask = np.zeros_like(mask_in)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_in, connectivity=8)
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] > Side_Contou_THD :
            mask[labels == i] = 255
            print(stats[i, cv2.CC_STAT_AREA])

    # cv2.imshow('img-mask-in2', mask)
            
    return mask
            
def enhance_split_detin(rdentin_bin):
    
    num_labels, labels = cv2.connectedComponents(rdentin_bin)

    for i in range(1, num_labels):
    
        component_mask = np.uint8(labels == i) * 255
        mask_rgb = cv2.cvtColor(component_mask, cv2.COLOR_GRAY2RGB)
        
        height, width = component_mask.shape[:2]
        # Define Translation Matrix
        M = np.float32([[1, 0, Side_Shift], [0, 1, 0]])
        # Use WarpAffine for x axis shift
        shifted_mask = cv2.warpAffine(component_mask, M, (width, height))
        diff_mask1 = np.int16(component_mask) - np.int16(shifted_mask)
        diff_mask1[diff_mask1 < 0] = 0
        diff_mask1 = np.uint8(diff_mask1)
        diff_mask1 = process_mask(diff_mask1)
        
        # Define Translation Matrix
        M = np.float32([[1, 0, -Side_Shift], [0, 1, 0]])
        # Use WarpAffine for x axis shift
        shifted_mask = cv2.warpAffine(component_mask, M, (width, height))
        diff_mask2 = np.int16(component_mask) - np.int16(shifted_mask)
        diff_mask2[diff_mask2 < 0] = 0
        diff_mask2 = np.uint8(diff_mask2)
        diff_mask2 = process_mask(diff_mask2)
        
        #Dilate to Enhance Feature
        kernel = np.ones((3, 3), np.uint8)
        diff_mask1 = cv2.dilate(diff_mask1, kernel, iterations=2)
        diff_mask2 = cv2.dilate(diff_mask2, kernel, iterations=2)

        # Get and Draw Min Rect.
        contours, _ = cv2.findContours(component_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contou = contours[0]
        rec_x, rec_y, rec_w, rec_h = cv2.boundingRect(contou)
        rect = cv2.minAreaRect(contou)
        boxp = cv2.boxPoints(rect)
        boxp = np.int0(boxp)
        # print(boxp)
        
        cv2.drawContours(mask_rgb, [boxp], 0, (0,255,0), 3)
        # cv2.imshow('mask_rgb', mask_rgb)

        
        mask_inter = mask_intersec(diff_mask2, diff_mask1, rec_y, rec_h, component_mask)
        
        
        comb_mask_or = cv2.bitwise_or(diff_mask1, diff_mask2)
#         comb_mask_and = cv2.bitwise_and(diff_mask1, diff_mask2)
        
        pair_rect_x = get_pair_rect(mask_inter, component_mask)
        
        mk_divide = mask_divide(pair_rect_x, mask_inter, rdentin_bin)
    
        # cv2.imshow('img-mask-org', component_mask)
        # cv2.imshow('img-diff1', diff_mask1)
        # cv2.imshow('img-diff2', diff_mask2)
        # cv2.imshow('img-comb-or', comb_mask_or)
        # cv2.imshow('mask-intersec', mask_inter)
        # cv2.imshow('mask-divide', mk_divide)
        
        # cv2.waitKey(0)
        
    # cv2.destroyAllWindows()

In [4]:
def main():
    # dir_input = './teeth3/1_ovlp_test' # 原始資料夾路徑
    # dir_input = './teeth3/300_ovlp' # 原始資料夾路徑
    dir_input = './teeth3/100_test'
    dir_output = './teeth3/100_test_out'
    
    dir_list = os.listdir(dir_input)
    
    all_true_stages = []
    all_predicted_stages = []
    
    for target_dir in dir_list:
        predictions = []
        dir_path = os.path.join(dir_input, target_dir)
        if not os.path.isdir(dir_path):
            continue
        # 定義Mask圖片和原始圖片的path
        gum_mask_path = os.path.join(dir_path, f"gum_{target_dir}.png")
        teeth_mask_path = os.path.join(dir_path, f"teeth_{target_dir}.png")
        dental_crown_path = os.path.join(dir_path, f"dentalcrown_{target_dir}.png")
        crown_path = os.path.join(dir_path, f"crown_{target_dir}.png")
        dentin_path = os.path.join(dir_path, f"dentin_{target_dir}.png")
        original_img_path = os.path.join(dir_path, f"raw_{target_dir}.png")
        correct_df = pd.read_excel(os.path.join(dir_path, f"analysis_{target_dir}.xlsx"))
        print(f"Processing {original_img_path} ...")

        # Load the images
        gum_img = cv2.imread(gum_mask_path, cv2.IMREAD_GRAYSCALE)
        teeth_img = cv2.imread(teeth_mask_path, cv2.IMREAD_GRAYSCALE)
        dental_crown_img = cv2.imread(dental_crown_path, cv2.IMREAD_GRAYSCALE)
        dentin_img = cv2.imread(dentin_path, cv2.IMREAD_GRAYSCALE)
        crown_img = cv2.imread(crown_path, cv2.IMREAD_GRAYSCALE)
        original_img = cv2.imread(original_img_path)

        # cv2.imshow('Detin Mask', dentin_img)
        # cv2.waitKey(0)
        
        # Threshold the images to binary
        _, gum_bin = cv2.threshold(gum_img, 128, 255, cv2.THRESH_BINARY)
        _, teeth_bin = cv2.threshold(teeth_img, 128, 255, cv2.THRESH_BINARY)
        _, dental_crown_bin = cv2.threshold(dental_crown_img, 128, 255, cv2.THRESH_BINARY)
        _, dentin_bin = cv2.threshold(dentin_img, 128, 255, cv2.THRESH_BINARY)
        _, crown_bin = cv2.threshold(crown_img, 128, 255, cv2.THRESH_BINARY)

        # 在Mask的圖片中找到輪廓
        contours_gum, _ = cv2.findContours(gum_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_teeth, _ = cv2.findContours(teeth_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_dental_crown, _ = cv2.findContours(dental_crown_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_dentin, _ = cv2.findContours(dentin_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # 複製原始圖片以進行標註
        image = original_img.copy()
        line_image = image.copy()

        # 定義膨脹內核
        kernel = np.ones((3, 3), np.uint8)

        # 獲取圖像尺寸
        height, width = dental_crown_bin.shape

        # 為未遮罩的區域創建遮罩
        non_masked_area = np.ones((height, width), dtype=np.uint8) * 255
        
        # 對牙冠Mask執行形態學操作
        dental_crown_bin = cv2.erode(dental_crown_bin, kernel, iterations=5)
        dental_crown_bin = cv2.dilate(dental_crown_bin, kernel, iterations=5)
        
        # 在牙冠Mask中找到連接的Components
        num_labels, labels = cv2.connectedComponents(dental_crown_bin)
        # 創建一個陣列來計算每個標籤的像素數
        label_counts = np.bincount(labels.flatten())

        # 設定閾值，僅保留像素數大於該值的區域
        pixel_threshold = 2000  # 可根據需求調整閾值

        # 創建一個新的二值圖像
        filtered_image = np.zeros_like(gum_bin)

        # 保留像素數大於閾值的區域
        for label in range(1, num_labels):
            print(label_counts[label])
            if label_counts[label] > pixel_threshold:
                filtered_image[labels == label] = 255

        dental_crown_bin = filtered_image
        n_kernel = np.ones((30, 1), np.uint8)

        # 處理 dentin_bin 雜點
        dentin_bin = cv2.erode(dentin_bin, n_kernel, iterations=1)
        dentin_bin = cv2.dilate(dentin_bin, n_kernel, iterations=1)
        gum_bin = cv2.dilate(gum_bin, n_kernel, iterations=2)
        gum_bin = cv2.erode(gum_bin, n_kernel, iterations=2)
        num_labels, labels = cv2.connectedComponents(gum_bin)

        # 創建一個陣列來計算每個標籤的像素數
        label_counts = np.bincount(labels.flatten())

        # 找出最大區域的標籤 (忽略背景標籤 0)
        max_label = np.argmax(label_counts[1:]) + 1

        # 創建一個新的二值圖像，只保留最大區域
        largest_component = np.zeros_like(gum_bin)
        largest_component[labels == max_label] = 255
        gum_bin = largest_component
        
        # 膨脹 gum
        dilated_gum_bin = cv2.dilate(gum_bin, kernel, iterations=10)
        num_labels, labels = cv2.connectedComponents(gum_bin)
        # 創建一個陣列來計算每個標籤的像素數
        label_counts = np.bincount(labels.flatten())

        # 找出最大區域的標籤 (忽略背景標籤 0)
        max_label = np.argmax(label_counts[1:]) + 1

        # 創建一個新的二值圖像，只保留最大區域
        largest_component = np.zeros_like(gum_bin)
        largest_component[labels == max_label] = 255
        gum_bin = largest_component
        
        # Combine all masks using bitwise operations
        combined_mask = cv2.bitwise_or(dilated_gum_bin, teeth_bin)
        combined_mask = cv2.bitwise_or(combined_mask, dental_crown_bin)
        combined_mask = cv2.bitwise_or(combined_mask, dentin_bin)

        # Invert the combined mask to get the non-masked areas
        non_masked_area = cv2.bitwise_not(combined_mask)
        # 膨脹 mask
        dilated_non_masked_area = cv2.dilate(non_masked_area, kernel, iterations=10)

        cv2.imwrite(os.path.join(dir_output, f"org_mask_{target_dir}.png"), dentin_bin) # alan test
        enhance_split_detin(dentin_bin) # alan test
        cv2.imwrite(os.path.join(dir_output, f"divide_{target_dir}.png"), dentin_bin) # alan test
        continue # alan test
        
        # 獲取連通區域
        num_labels, labels = cv2.connectedComponents(dentin_bin)

        # 將 mask 以帶有透明度的方式疊加在原始圖像上
        overlay = image.copy()
        overlay[dental_crown_bin > 0] = (163, 118, 158)  # 將 dental_crown 顯示
        overlay[dentin_bin > 0] = (117, 122, 152)  # 將 dentin 顯示
        overlay[gum_bin > 0] = (0, 177, 177)  # 將 dentin 顯示
        overlay[crown_bin > 0] = (255, 0, 128)# 將 crown 顯示


        # 為每個區域創建一個獨立的 mask，進行膨脹並標註最左及最右交點
        for i in range(1, num_labels):  # 從1開始，0是背景

            component_mask = np.uint8(labels == i) * 255
            
            area = cv2.countNonZero(component_mask)
            if area < 500:
                #print("Skip, area = " ,area)
                continue
            #print(area)
            contours, _ = cv2.findContours(component_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            rect = cv2.minAreaRect(contours[0])
            box = cv2.boxPoints(rect)
            box = np.int64(box)
            
            
       

In [5]:
if __name__ == "__main__":
    main()

Processing ./teeth3/100_test\1\raw_1.png ...
69551
68777
114
432
1896
873
507
804
135
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
find a pair rect. st. x = []
423
993
1038
585
1842
774
111
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1